In [ ]:
from polygon import RESTClient
import pandas as pd
import asyncio
import logging
from datetime import datetime, timedelta
import pytz
from typing import Dict, List, Optional

class PolygonMarketScanner:
    def __init__(
        self, 
        api_key: str,
        min_market_cap: float = 300e6,
        min_avg_volume: int = 100000,
        gap_threshold: float = 2.0,
    ):
        self.client = RESTClient(api_key)
        self.logger = logging.getLogger('PolygonScanner')
        self.logger.setLevel(logging.INFO)
        self.min_market_cap = min_market_cap
        self.min_avg_volume = min_avg_volume
        self.gap_threshold = gap_threshold
        self.semaphore = asyncio.Semaphore(200)
        self.processed_count = 0
        self.total_tickers = 0

    def _get_filtered_tickers(self) -> List[Dict]:
        """Get pre-filtered list of potential candidates"""
        print("Pre-filtering tickers...")
        candidates = []
        
        # Get base ticker list
        base_tickers = self.client.list_tickers(
            market="stocks",
            type="CS",
            active=True,
            limit=1000
        )
        
        # Get yesterday's date
        yesterday = (datetime.now() - timedelta(days=1)).strftime('%Y-%m-%d')
        
        for ticker in base_tickers:
            if ticker.primary_exchange not in ['XNYS', 'XNAS']:
                continue
                
            try:
                # Get ticker details first
                details = self.client.get_ticker_details(ticker.ticker)
                if not details or not details.market_cap or details.market_cap < self.min_market_cap:
                    continue
                
                # Quick volume check
                daily_aggs = list(self.client.list_aggs(
                    ticker.ticker,
                    multiplier=1,
                    timespan="day",
                    from_=yesterday,
                    to=yesterday,
                    limit=1
                ))
                
                if daily_aggs and daily_aggs[0].volume >= self.min_avg_volume:
                    candidates.append({
                        'symbol': ticker.ticker,
                        'market_cap': details.market_cap,
                        'name': details.name,
                        'prev_close': daily_aggs[0].close,
                        'prev_volume': daily_aggs[0].volume
                    })
                    
                    print(f"Pre-filtered: {ticker.ticker} | Cap: ${details.market_cap/1e9:.1f}B | "
                          f"Vol: {daily_aggs[0].volume:,}", end='\r')
                
            except Exception as e:
                continue
        
        print(f"\nFound {len(candidates)} potential candidates after pre-filtering")
        return candidates

    async def get_pre_market_data(self, symbol: str, prev_data: Dict, start_date: str) -> Optional[Dict]:
        """Get pre-market data for pre-filtered candidates"""
        try:
            async with self.semaphore, asyncio.timeout(1):
                # Get only pre-market data
                aggs = list(self.client.list_aggs(
                    ticker=symbol,
                    multiplier=1,
                    timespan="minute",
                    from_=start_date,
                    to=start_date,
                    limit=50000
                ))
                
                if not aggs:
                    return None

                current_price = aggs[-1].close
                current_volume = sum(agg.volume for agg in aggs)
                
                # Quick gap calculation
                gap_percent = ((current_price - prev_data['prev_close']) / 
                             prev_data['prev_close'] * 100)
                
                if gap_percent <= self.gap_threshold:
                    return None

                # Market cap categorization
                market_cap = prev_data['market_cap']
                if market_cap >= 200e9:
                    cap_type = 'Titans'
                elif market_cap >= 10e9:
                    cap_type = 'Large caps'
                elif market_cap >= 2e9:
                    cap_type = 'Mid caps'
                elif market_cap >= 300e6:
                    cap_type = 'Small caps'
                elif market_cap > 50e6:
                    cap_type = 'Micro caps'
                else:
                    cap_type = 'Shrimp'

                result = {
                    'symbol': symbol,
                    'name': prev_data['name'],
                    'market_cap': market_cap,
                    'market_cap_type': cap_type,
                    'gap_percent': round(gap_percent, 2),
                    'price': current_price,
                    'volume': current_volume,
                    'prev_close': prev_data['prev_close'],
                    'prev_volume': prev_data['prev_volume']
                }
                
                print(f"\nFound Gapper: {symbol:<6} | Gap: {gap_percent:>5.1f}% | "
                      f"{cap_type:<10} | ${market_cap/1e9:>6.1f}B | "
                      f"Vol: {current_volume:>10,}")
                
                return result

        except Exception as e:
            return None

    async def scan_pre_market(self) -> pd.DataFrame:
        start_time = datetime.now()
        print("\nStarting pre-market scan...")
        
        # Get pre-filtered candidates
        candidates = self._get_filtered_tickers()
        
        if not candidates:
            print("No candidates found after pre-filtering")
            return pd.DataFrame()
        
        # Get today's date
        today = datetime.now(pytz.timezone('US/Eastern')).strftime('%Y-%m-%d')
        
        # Process pre-filtered candidates
        tasks = [
            self.get_pre_market_data(
                candidate['symbol'], 
                candidate, 
                today
            ) 
            for candidate in candidates
        ]
        
        results = await asyncio.gather(*tasks)
        results = [r for r in results if r is not None]
        
        # Convert to DataFrame
        df = pd.DataFrame(results) if results else pd.DataFrame()
        
        if not df.empty:
            df = df.sort_values('gap_percent', ascending=False)
        
        execution_time = (datetime.now() - start_time).total_seconds()
        print(f"\nScan completed in {execution_time:.2f} seconds.")
        print(f"Found {len(df)} matching stocks")
        
        if not df.empty:
            print("\nTop 5 Pre-Market Gainers:")
            print(df.head().to_string())
        
        return df

async def get_pre_market_gainers(api_key: str) -> pd.DataFrame:
    scanner = PolygonMarketScanner(api_key)
    return await scanner.scan_pre_market()

In [8]:
import dotenv
import os
dotenv.load_dotenv()

api_key = os.getenv('polygonioKEY')

In [ ]:
poly_preg_df = await get_pre_market_gainers(api_key)

Running in after-hours mode

Starting after-hours scan...
Fetching ticker list...
Processing 4979 tickers...

Found: AACT   | Change: $  0.00 |   0.1% | Vol: 1,604 | Session: after-hours

Found: AADI   | Change: $  0.06 |   2.9% | Vol: 71,365 | Session: after-hours

Found: AAON   | Change: $  2.86 |   2.1% | Vol: 327,370 | Session: after-hours

Found: AAP    | Change: $  0.08 |   0.2% | Vol: 1,825,539 | Session: after-hours

Found: AAPL   | Change: $  2.66 |   1.1% | Vol: 37,586,535.0 | Session: after-hours

Found: AAT    | Change: $  0.38 |   1.3% | Vol: 152,713 | Session: after-hours

Found: AB     | Change: $  0.49 |   1.4% | Vol: 415,768 | Session: after-hours

Found: ABBV   | Change: $  4.44 |   2.5% | Vol: 5,452,743 | Session: after-hours

Found: ABEO   | Change: $  0.09 |   1.5% | Vol: 197,126 | Session: after-hours

Found: ABL    | Change: $  0.18 |   2.4% | Vol: 664,916 | Session: after-hours

Found: ABLLL  | Change: $  0.09 |   0.3% | Vol: 2,167 | Session: after-hours

Found:

In [4]:
print(len(poly_preg_df))

0
